### Парсинг HTML. BeautifulSoup, MongoDB

#### Задание 1

In [1]:
from bs4 import BeautifulSoup as bs
import requests as rq
from pprint import pprint
import pandas as pd
import re

#### HH

In [2]:
main_link_hh = 'https://spb.hh.ru'
header_hh = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.109 Safari/537.36',
          'Accept':'*/*'}
params_hh = {'clusters':True,
          'enable_snippets':True,
          'text':'Data scientist',
          'L_save_area':True,
          'area':113,
          'from':'cluster_area',
          'showClusters':True}

In [3]:
response_hh = rq.get(main_link_hh + '/search/vacancy', headers=header_hh, params=params_hh).text
soup_hh = bs(response_hh, 'lxml')
vacancy_block_hh = soup_hh.find('div',{'class':'vacancy-serp'})
vacancy_list_hh = vacancy_block_hh.find_all('div',{'class':'vacancy-serp-item'})

In [4]:
pages_hh = []
while True:
    if soup_hh.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'}):
        next_page_hh = main_link_hh + soup_hh.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})['href']
    else:
        break
    pages_hh.append(next_page_hh)
    response_hh = rq.get(next_page_hh, headers=header_hh).text
    soup_hh = bs(response_hh, 'lxml')
    vacancy_block_hh_ = soup_hh.find('div',{'class':'vacancy-serp'})
    vacancy_list_hh_ = vacancy_block_hh_.find_all('div',{'class':'vacancy-serp-item'})
pprint(pages_hh)

['https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true&text=Data+scientist&page=1',
 'https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true&text=Data+scientist&page=2',
 'https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true&text=Data+scientist&page=3',
 'https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true&text=Data+scientist&page=4',
 'https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true&text=Data+scientist&page=5']


#### SuperJob

In [5]:
main_link_sj = 'https://russia.superjob.ru'
header_sj = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.109 Safari/537.36',
          'Accept':'*/*'}
params_sj = {'keywords':'Data'}

In [6]:
response_sj = rq.get(main_link_sj + '/vacancy/search', headers=header_sj, params=params_sj).text
soup_sj = bs(response_sj, 'lxml')
vacancy_block_sj = soup_sj.find('div',{'class':'_1ID8B'})
vacancy_list_sj = vacancy_block_sj.find_all('div',{'class':'iJCa5 f-test-vacancy-item _1fma_ _1JhPh _2gFpt _1znz6 _2nteL'})

In [7]:
pages_sj = []
while True:
    if soup_sj.find('a',{'class':'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-Dalshe'}):
        next_page_sj = main_link_sj + soup_sj.find('a',{'class':'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-Dalshe'})['href']
    else:
        break
    pages_sj.append(next_page_sj)
    response_sj = rq.get(next_page_sj, headers=header_sj).text
    soup_sj = bs(response_sj, 'lxml')
    vacancy_block_sj_ = soup_sj.find('div',{'class':'_1ID8B'})
    vacancy_list_sj_ = vacancy_block_sj_.find_all('div',{'class':'iJCa5 f-test-vacancy-item _1fma_ _1JhPh _2gFpt _1znz6 _2nteL'})
pprint(pages_sj)

['https://russia.superjob.ru/vacancy/search/?keywords=Data&page=2',
 'https://russia.superjob.ru/vacancy/search/?keywords=Data&page=3']


In [8]:
jobs_hh = []

In [9]:
for job_hh in vacancy_list_hh + vacancy_list_hh_:
    
    job_data_hh = {}
    
    job_name_hh = job_hh.find('div',{'class':'vacancy-serp-item__info'}).getText()
    job_company_hh = job_hh.find('a',{'class':'bloko-link bloko-link_secondary'}).getText()
    job_city_hh = job_hh.find('span',{'class':'vacancy-serp-item__meta-info'}).getText().split()[0].replace(',', '')
      
    # Зарплата HH:
    job_salary_hh = job_hh.find('div',{'class':'vacancy-serp-item__sidebar'}).getText().replace('\xa0', '')
    min_salary_reg_hh = '^\d*\s\d*|^[от]+\s(\d*\s\d*)'
    max_salary_reg_hh = '-(\d*\s\d*)|^[до]+\s(\d*\s\d*)'
    currency_reg_hh = '([a-z]*)+$|([а-я]*)\.+$'
    min_salary_pattern_hh = re.compile(min_salary_reg_hh)
    max_salary_pattern_hh = re.compile(max_salary_reg_hh)
    currency_pattern_hh = re.compile(currency_reg_hh, re.I)
    min_salary_hh = min_salary_pattern_hh.search(job_salary_hh)
    max_salary_hh = max_salary_pattern_hh.search(job_salary_hh)
    currency_hh = currency_pattern_hh.search(job_salary_hh)
    if min_salary_hh:
        min_salary_hh = min_salary_hh.group(0).replace('от', '')
    if max_salary_hh:
        max_salary_hh = max_salary_hh.group(0).replace('-', '')
    if currency_hh:
        currency_hh = currency_hh.group(0).replace('.', '')
    #
        
    job_link_hh = job_hh.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
        
    job_data_hh['name'] = job_name_hh
    job_data_hh['company'] = job_company_hh
    job_data_hh['city'] = job_city_hh
    job_data_hh['min_salary'] = min_salary_hh
    job_data_hh['max_salary'] = max_salary_hh
    job_data_hh['currency'] = currency_hh
    job_data_hh['link'] = job_link_hh
    job_data_hh['from'] = main_link_hh
    
    jobs_hh.append(job_data_hh)

In [10]:
jobs_sj = []

In [11]:
for job_sj in vacancy_list_sj + vacancy_list_sj_:
    
    job_data_sj = {}
    
    job_name_sj = job_sj.find('div',{'class':'_3mfro PlM3e _2JVkc _3LJqf'}).getText()
    job_company_sj = job_sj.find('span',{'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _2VHxz _15msI'}).getText()
    job_city_sj = job_sj.find('span',{'class':'_3mfro f-test-text-company-item-location _9fXTd _2JVkc _2VHxz'}).getText().split()[-1]
    
    # Зарплата SJ:
    job_salary_sj = job_sj.find('span',{'class':'_1OuF_ _1qw9T f-test-text-company-item-salary'}).getText().replace('\xa0', '').replace('/месяц', '')
    min_salary_reg_sj = '-(\d*\d*)|^[от]+(\d*\d*)'
    max_salary_reg_sj = '-(\d*\d*)|^[до]+(\d*\d*)'
    currency_reg_sj = '([a-z]*)+$|([а-я]*)\.+$'
    min_salary_pattern_sj = re.compile(min_salary_reg_sj)
    max_salary_pattern_sj = re.compile(max_salary_reg_sj)
    currency_pattern_sj = re.compile(currency_reg_sj, re.I)
    min_salary_sj = min_salary_pattern_sj.search(job_salary_sj)
    max_salary_sj = max_salary_pattern_sj.search(job_salary_sj)
    currency_sj = currency_pattern_sj.search(job_salary_sj)
    if min_salary_sj:
        min_salary_sj = min_salary_sj.group(0).replace('от', '').replace('о', 'None')
    if max_salary_sj:
        max_salary_sj = max_salary_sj.group(0).replace('до', '').replace('о', 'None')
    if currency_sj:
        currency_sj = currency_sj.group(0).replace('.', '')
    #
        
    job_link_sj = main_link_sj + job_sj.find('a',{'class':'icMQ_'})['href']
     
    job_data_sj['name'] = job_name_sj
    job_data_sj['company'] = job_company_sj
    job_data_sj['city'] = job_city_sj
    job_data_sj['min_salary'] = min_salary_sj
    job_data_sj['max_salary'] = max_salary_sj
    job_data_sj['currency'] = currency_sj
    job_data_sj['link'] = job_link_sj
    job_data_sj['from'] = main_link_sj
    
    jobs_sj.append(job_data_sj)

In [12]:
pd.DataFrame(jobs_hh + jobs_sj)

,name,company,city,min_salary,max_salary,currency,link,from
0,Data Scientist,ООО КРИПТОН,Москва,None,180000,руб,https://spb.hh.ru/vacancy/37635578?query=Data%...,https://spb.hh.ru
1,Python Data Scientist,Web Interactive World,Москва,150000,None,руб,https://spb.hh.ru/vacancy/37521387?query=Data%...,https://spb.hh.ru
2,Аналитик-математик (Data scientist),ООО Финансово-правовой альянс,Санкт-Петербург,None,100000,руб,https://spb.hh.ru/vacancy/37698748?query=Data%...,https://spb.hh.ru
3,Data Scientist,ООО Варити,Москва,130000,None,руб,https://spb.hh.ru/vacancy/37569615?query=Data%...,https://spb.hh.ru
4,Data Scientist,Корпорация WCH Service Bureau,Казань,None,None,,https://spb.hh.ru/vacancy/37662353?query=Data%...,https://spb.hh.ru
...,...,...,...,...,...,...,...,...
115,Senior Browser Operation Manager,Коулмэн Сервис,Москва,None,None,,https://russia.superjob.ru/vakansii/senior-bro...,https://russia.superjob.ru
116,"Sales manager (B2B, Russia and CIS)",Коулмэн Сервис,Москва,None,None,,https://russia.superjob.ru/vakansii/sales-mana...,https://russia.superjob.ru
117,IOS Team lead / Senior IOS Developer,Antal Russia,Москва,None,None,,https://russia.superjob.ru/vakansii/ios-team-l...,https://russia.superjob.ru
118,Android Team Lead / Senior Android Developer,Antal Russia,Москва,None,None,,https://russia.superjob.ru/vakansii/android-te...,https://russia.superjob.ru
